In [366]:
import feather
import scipy
import matplotlib.pyplot as plt
import sys

%load_ext autoreload

sys.path.append('../iw_inverse')
import iw_invert as iwi

%autoreload 2

strat = feather.read_dataframe('sbc_strat.fthr')
df    = feather.read_dataframe('sbc2.fthr')
df['datetime'] = pd.to_datetime(df['time'],unit='s')
df = df.set_index('datetime')

df = df.loc[df.x == 0]



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [432]:
#DP = np.unique(df['z'])
DP = strat['depth'].values
DP = DP[np.arange(0,len(DP),10)]
N2 = np.interp(DP, strat['depth'],strat['strat'])
M2 = 1/12.42
#FREQS = np.array([M2])/3600
FREQS = np.array( [0.79822616, 
                   M2,
                   1.19733925,  
                   1.99556541 ,  
                   8.7804878 ,  
                   60.26607539,
                   64.25720621 , 
                   96.18625277, 
                   107.76053215, 
                   205.94235033])/3600 


MODES = np.arange(1,10)

#Dataframe subsample 
#df = df.loc[ df['z']  > 150]

dfss_times = df.resample('1H').first()
dfss = df.loc[ df['time'].isin(dfss_times.time) ]

inv = iwi.InternalWaveInversion(dfss,FREQS,MODES,DP,N2,tc='pdz')
inv.pinvert()
inv.ps



,freqs,modes,hwvn,a,b
0,0.000222,1,0.001003+0.000000j,-23.331956+0.000000j,-6.395737+0.000000j
1,0.000222,2,0.001539+0.000000j,24.277380+0.000000j,-1.095126+0.000000j
2,0.000222,3,0.002073+0.000000j,-5.107394+0.000000j,81.093670+0.000000j
3,0.000222,4,0.002618+0.000000j,4.693012+0.000000j,-41.224773+0.000000j
4,0.000222,5,0.003139+0.000000j,9.341180+0.000000j,-39.960992+0.000000j
...,...,...,...,...,...
85,0.057206,5,0.000000-0.031420j,30.015560+0.000000j,9.001385+0.000000j
86,0.057206,6,0.000000-0.036644j,-0.670386+0.000000j,57.513892+0.000000j
87,0.057206,7,0.000000-0.041863j,4.548049+0.000000j,23.481746+0.000000j
88,0.057206,8,0.000000-0.047074j,64.606594+0.000000j,-7.089849+0.000000j


In [433]:
dz_hat = inv.H @ np.concatenate([inv.ps['a'],inv.ps['b']]).T


In [434]:
%matplotlib widget
import cmocean

def plot_dataframe(df,col,ax):
    zz,tt = np.meshgrid(np.unique(df.z), np.unique(df.index))
    out = np.array(df[col]).reshape(zz.shape)
    ax.invert_yaxis()
    p = ax.contourf(tt,zz,out,levels=np.linspace(-1e4,1e4),cmap='seismic')
    return p
    
    
def plot_estimate(df,est,ax):
    zz,tt = np.meshgrid(np.unique(df.z), np.unique(df.index))
    out = np.array(est.reshape(zz.shape) )
    ax.invert_yaxis()
    p = ax.contourf(tt,zz,out,levels=np.linspace(-1e4,1e4),cmap='seismic')
    return p
    return p

def plot_anomaly(df,col,est,ax):
    zz,tt = np.meshgrid(np.unique(df.z), np.unique(df.index))
    out1 = np.array(est.reshape(zz.shape) )
    out2 = np.array(df[col]).reshape(zz.shape)
    out3 = out2-out1
    ax.invert_yaxis()
    p = ax.contourf(tt,zz,out3,levels=np.linspace(-1e4,1e4),cmap='seismic')
    return p


def plot_rms_error(df,col,est,ax):
    zz,tt = np.meshgrid(np.unique(df.z), np.unique(df.index))
    dd = np.array(df[col]).reshape(zz.shape)
    ee = np.array(est.reshape(zz.shape) )
    diff = ee - dd
    rme = np.sqrt ( np.mean(diff**2,axis=0)  / np.mean( dd**2 ,axis=0 ) )
    p = ax.plot(zz[0,:],rme,'k.--')
    ax.set_xlabel('Depth')
    return p
    
    
f , ax = plt.subplots(2,2)
p = plot_dataframe(dfss,'pdz' , ax[0,0])
p = plot_estimate(dfss,dz_hat, ax[0,1])
p = plot_anomaly(dfss,'pdz',dz_hat, ax[1,0])
plot_rms_error(dfss,'pdz',dz_hat,ax[1,1])

f.autofmt_xdate()
cbar_ax = f.add_axes([0.92, 0.2, 0.02, 0.7])
ax[0,0].set_title('Data')
ax[0,1].set_title('Model')
ax[1,0].set_title('Error')
ax[1,1].set_title('Relative RMS Error')

f.set_size_inches(10,7) 
f.colorbar(p,cax=cbar_ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jpdef/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:2786: ComplexWarning: Casting complex values to real discards the imaginary part
  order=order, subok=True, ndmin=ndmin)
/home/jpdef/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [428]:
%matplotlib widget
def mag(a,b):
    return np.abs(a + b)

def plot_params(df,ax):
    mm,ff = np.meshgrid(np.unique(df.modes), np.unique(df.freqs)*3600)
    aa = np.array(df['a']).reshape(mm.shape) 
    bb = np.array(df['b']).reshape(mm.shape) 
    p = ax.pcolormesh(ff,mm,(mag(aa,bb)))#levels=np.linspace(-1e4,1e4),cmap='seismic')
    return p

f ,ax = plt.subplots()
p = plot_params(inv.ps,ax)
ax.set_ylabel('modes')
ax.set_xlabel('cph')
f.colorbar(p)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …